ml 02.11

 ⁃ bias variance 

сложный датасет с нелин данными и большим обьемом

анал данных: 
 1. пропуски, 
 2. константные признаки, 
 3. корреляция
 4. zero split method (для всех признаков строим любой максимально глубокий бустинг с подбором параметров (много деревьев глубоких); после выводим feature importance и удаляем с нулевым.
 5. для каждой пары скоррелированных признаков выводим корреляцию с таргетом и удаляем тот, у которого меньше.
 6. все три вида бустинга lightGBM, XGBoost, CatBoost (из оф. библиотек). Для каждого используем любой удобный подбор параметров на валидации(70+15+15). Тестим на данных.
 7. Проверка всех метрик: acc, rec, pr, roc auc, поиграться с cut off (построить кривую pr|rec и поток)

In [45]:
#!pip install kaggle

In [46]:
#!kaggle competitions download -c ieee-fraud-detection
#!python -m venv sklearn-env
#!sklearn-env\Scripts\activate с
#!pip install -U scikit-learn

In [47]:
import os, gc, math, json, warnings
warnings.filterwarnings('ignore')

In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [49]:
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    roc_auc_score, roc_curve
)
from sklearn.model_selection  import KFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [50]:
#!pip install lightgbm

In [51]:
#!pip install xgboost

In [52]:
#!pip install catboost

In [53]:
#!pip install category-encoders

In [54]:
import category_encoders as ce

In [55]:
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier, Pool

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [56]:
DATA_DIR = "ieee-fraud-detection"

train_tr_path = os.path.join(DATA_DIR, "train_transaction.csv")
train_id_path = os.path.join(DATA_DIR, "train_identity.csv")
test_tr_path = os.path.join(DATA_DIR, "test_transaction.csv")
test_id_path = os.path.join(DATA_DIR, "test_identity.csv")
sub_path = os.path.join(DATA_DIR, "sample_submission.csv")


In [57]:
def reduce_mem_usage(df, verbose=True):
    
    # считаем, сколько памяти юзаем (deep=True учитывает размер самих значений,
    # а не только контейнеров)
    start_mem = df.memory_usage(deep=True).sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtype
        if col_type.kind in ['i','u','f']:
            c_min = df[col].min()
            c_max = df[col].max()
            if col_type.kind in ['i','u']:
                if c_min >= 0:
                    if c_max < 255:
                        df[col] = df[col].astype(np.uint8)
                    elif c_max < 65535:
                        df[col] = df[col].astype(np.uint16)
                    elif c_max < 4294967295:
                        df[col] = df[col].astype(np.uint32)
                    else:
                        df[col] = df[col].astype(np.uint64)
                else:
                    if np.iinfo(np.int8).min < c_min < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif np.iinfo(np.int16).min < c_min < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif np.iinfo(np.int32).min < c_min < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    else:
                        df[col] = df[col].astype(np.int64)
            else:
                df[col] = pd.to_numeric(df[col], downcast='float')
        elif col_type == 'object':
            # не переводим автоматически в category, тк хз точно ли категориальные признаки
            pass
    end_mem = df.memory_usage(deep=True).sum() / 1024**2
    if verbose:
        print(f"Mem. {start_mem:.2f} MB → {end_mem:.2f} MB")
    return df

def read_csv_safely(path):
    # Вариант с dtype=None, чтобы дать Pandas самому определить, затем downcast
    df = pd.read_csv(path)
    return reduce_mem_usage(df)


In [58]:
train_tr = read_csv_safely(train_tr_path)
train_id = read_csv_safely(train_id_path)
test_tr  = read_csv_safely(test_tr_path)
test_id  = read_csv_safely(test_id_path)

train = train_tr.merge(train_id, how='left', on='TransactionID')
test  = test_tr.merge(test_id,  how='left', on='TransactionID')

print("train:", train.shape, "test:", test.shape)
train.head(3)

Mem. 2062.07 MB → 1203.22 MB
Mem. 143.14 MB → 129.94 MB
Mem. 1771.84 MB → 1038.31 MB
Mem. 140.08 MB → 127.09 MB
train: (590540, 434) test: (506691, 433)


,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
train.describe()

,TransactionID,isFraud,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2,...,id_17,id_18,id_19,id_20,id_21,id_22,id_24,id_25,id_26,id_32
count,5.905400e+05,590540.000000,5.905400e+05,590540.000000,590540.000000,581607.000000,588975.000000,586281.000000,524834.000000,524834.000000,...,139369.000000,45113.000000,139318.000000,139261.000000,5159.000000,5169.000000,4747.000000,5132.000000,5163.000000,77586.000000
mean,3.282270e+06,0.034990,7.372311e+06,135.027161,9898.734658,362.555511,153.194946,199.278900,290.733826,86.800652,...,189.451370,14.237337,353.128174,403.882568,368.269806,16.002708,12.800927,329.608917,149.070312,26.508596
std,1.704744e+05,0.183755,4.617224e+06,239.162689,4901.170153,157.817963,11.343591,41.332325,101.700386,2.773530,...,30.377136,1.561116,141.112030,152.158493,198.849014,6.897755,2.372468,97.462585,32.101933,3.737366
min,2.987000e+06,0.000000,8.640000e+04,0.251000,1000.000000,100.000000,100.000000,100.000000,100.000000,10.000000,...,100.000000,10.000000,100.000000,100.000000,100.000000,10.000000,11.000000,100.000000,100.000000,0.000000
25%,3.134635e+06,0.000000,3.027058e+06,43.320999,6019.000000,214.000000,150.000000,166.000000,204.000000,87.000000,...,166.000000,13.000000,266.000000,256.000000,252.000000,14.000000,11.000000,321.000000,119.000000,24.000000
50%,3.282270e+06,0.000000,7.306528e+06,68.769001,9678.000000,361.000000,150.000000,226.000000,299.000000,87.000000,...,166.000000,15.000000,341.000000,472.000000,252.000000,14.000000,11.000000,321.000000,149.000000,24.000000
75%,3.429904e+06,0.000000,1.124662e+07,125.000000,14184.000000,512.000000,150.000000,226.000000,330.000000,87.000000,...,225.000000,15.000000,427.000000,533.000000,486.500000,14.000000,15.000000,371.000000,169.000000,32.000000
max,3.577539e+06,1.000000,1.581113e+07,31937.390625,18396.000000,600.000000,231.000000,237.000000,540.000000,102.000000,...,229.000000,29.000000,671.000000,661.000000,854.000000,44.000000,26.000000,548.000000,216.000000,32.000000


In [60]:
np.iinfo(np.int16)

iinfo(min=-32768, max=32767, dtype=int16)

In [61]:
target_col = 'isFraud'
print(train[target_col].mean())
miss = train.isna().mean().sort_values(ascending=False).head(10)
miss.to_frame('miss_ratio')

print('tot_cols: ', train.shape[1])
print('num_cols: ', train.select_dtypes(include=[np.number]).shape[1])
print('obj_cols: ', train.select_dtypes(include=['object']).shape[1])

cat_candidates = [c for c in train.columns if train[c].dtype == 'object']
cat_candidates

0.03499000914417313
tot_cols:  434
num_cols:  403
obj_cols:  31


['ProductCD',
 'card4',
 'card6',
 'P_emaildomain',
 'R_emaildomain',
 'M1',
 'M2',
 'M3',
 'M4',
 'M5',
 'M6',
 'M7',
 'M8',
 'M9',
 'id_12',
 'id_15',
 'id_16',
 'id_23',
 'id_27',
 'id_28',
 'id_29',
 'id_30',
 'id_31',
 'id_33',
 'id_34',
 'id_35',
 'id_36',
 'id_37',
 'id_38',
 'DeviceType',
 'DeviceInfo']

In [62]:

# TransactionDT — секунды с начала «нулевого» времени. Сделаем фичи:
def add_time_features(df):
    if 'TransactionDT' in df.columns:
        df['DT'] = df['TransactionDT']
        df['DT_day'] = (df['DT'] // (24*60*60)).astype('int32')
        df['DT_hour'] = ((df['DT'] // (60*60)) % 24).astype('int16')
        df['DT_dayofweek'] = (df['DT_day'] % 7).astype('int8')
    return df

def add_amount_features(df):
    if 'TransactionAmt' in df.columns:
        df['TransactionAmt_log1p'] = np.log1p(df['TransactionAmt'].astype(float))
    return df

def freq_encode(train, test, cols):
    for c in cols:
        fq = train[c].value_counts(dropna=False)
        train[c + '_fq'] = train[c].map(fq)
        test[c + '_fq']  = test[c].map(fq)
    return train, test

train = add_time_features(train)
test  = add_time_features(test)
train = add_amount_features(train)
test  = add_amount_features(test)

# Примеры частотных энкодингов для card1/addr1/emaildomain при наличии
freq_cols = [c for c in ['card1','addr1','P_emaildomain','R_emaildomain'] if c in train.columns]
train, test = freq_encode(train, test, freq_cols)


In [64]:

drop_cols = [target_col, 'TransactionID']

features = [c for c in train.columns if c not in drop_cols]

# Временной сплит по TransactionDT: последний хвост как валидация
cutoff = np.quantile(train['TransactionDT'], 0.85)  
trn_idx = train['TransactionDT'] < cutoff
val_idx = ~trn_idx

X_tr = train.loc[trn_idx, features].reset_index(drop=True)
y_tr = train.loc[trn_idx, target_col].astype('int8').reset_index(drop=True)
X_va = train.loc[val_idx, features].reset_index(drop=True)
y_va = train.loc[val_idx, target_col].astype('int8').reset_index(drop=True)

X_te = test[features].reset_index(drop=True)

print(X_tr.shape, X_va.shape, X_te.shape)


KeyError: "['id_01', 'id_02', 'id_03', 'id_04', 'id_05', 'id_06', 'id_07', 'id_08', 'id_09', 'id_10', 'id_11', 'id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29', 'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38'] not in index"

In [ ]:

# Эвристика: object → категориальные
cat_cols = [c for c in features if train[c].dtype == 'object']
num_cols = [c for c in features if c not in cat_cols]

print("num_cols:", len(num_cols), "cat_cols:", len(cat_cols))
cat_cols[:20]


In [ ]:

# Приводим object к category
def cast_category(df, cols):
    for c in cols:
        df[c] = df[c].astype('category')
    return df

X_tr_lgb = X_tr.copy()
X_va_lgb = X_va.copy()
X_te_lgb = X_te.copy()

X_tr_lgb = cast_category(X_tr_lgb, cat_cols)
X_va_lgb = cast_category(X_va_lgb, cat_cols)
X_te_lgb = cast_category(X_te_lgb, cat_cols)

lgb_train = lgb.Dataset(X_tr_lgb, label=y_tr, categorical_feature=cat_cols, free_raw_data=False)
lgb_valid = lgb.Dataset(X_va_lgb, label=y_va, categorical_feature=cat_cols, free_raw_data=False)

lgb_params = dict(
    objective='binary',
    metric='auc',
    learning_rate=0.05,
    num_leaves=256,
    max_depth=-1,
    subsample=0.8,
    colsample_bytree=0.8,
    min_data_in_leaf=64,
    reg_alpha=0.1,
    reg_lambda=0.1,
    verbose=-1,
    seed=RANDOM_STATE,
)

lgb_model = lgb.train(
    lgb_params,
    lgb_train,
    num_boost_round=10000,
    valid_sets=[lgb_train, lgb_valid],
    valid_names=['train','valid'],
    early_stopping_rounds=200,
    verbose_eval=200
)

va_pred_lgb = lgb_model.predict(X_va_lgb, num_iteration=lgb_model.best_iteration)
print("LightGBM AUC:", roc_auc_score(y_va, va_pred_lgb))
print("LightGBM PR-AUC:", average_precision_score(y_va, va_pred_lgb))


In [ ]:

# Важности признаков
imp = pd.DataFrame({
    'feature': lgb_model.feature_name(),
    'importance': lgb_model.feature_importance(importance_type='gain')
}).sort_values('importance', ascending=False).head(40)

plt.figure(figsize=(8,10))
plt.barh(imp['feature'].iloc[::-1], imp['importance'].iloc[::-1])
plt.title('LightGBM Feature Importance (gain, top-40)')
plt.tight_layout()
plt.show()


In [ ]:

# OOF Target Encoding для cat_cols
def oof_target_encode(X, y, X_valid, X_test, cols, n_splits=5, smoothing=20, random_state=RANDOM_STATE, add_noise=0.0):
    X = X.copy()
    X_valid = X_valid.copy()
    X_test = X_test.copy()

    oof = pd.DataFrame(index=X.index)
    te_models = []
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    for c in cols:
        oof[c] = np.nan

    for tr_idx, va_idx in kf.split(X):
        X_tr_f, X_va_f = X.iloc[tr_idx], X.iloc[va_idx]
        y_tr_f = y.iloc[tr_idx]

        te = ce.TargetEncoder(cols=cols, smoothing=smoothing)
        te.fit(X_tr_f, y_tr_f)
        oof.iloc[va_idx] = te.transform(X_va_f)[cols].values
        te_models.append(te)

    # финальный энкодер на полном трейне для теста/валидации
    te_full = ce.TargetEncoder(cols=cols, smoothing=smoothing)
    te_full.fit(X, y)
    X_valid_te = te_full.transform(X_valid)[cols]
    X_test_te  = te_full.transform(X_test)[cols]

    # шум для регуляризации
    if add_noise > 0:
        noise = np.random.normal(0, add_noise, size=oof[cols].shape)
        oof[cols] = oof[cols] + noise

    # добавим TE фичи в датасеты
    X_te_tr = X.copy()
    X_te_va = X_valid.copy()
    X_te_te = X_test.copy()
    for c in cols:
        X_te_tr[c + "_te"] = oof[c].astype(np.float32)
        X_te_va[c + "_te"] = X_valid_te[c].astype(np.float32)
        X_te_te[c + "_te"] = X_test_te[c].astype(np.float32)

    # можно удалить исходные категориальные колонки, чтобы оставить только TE-варианты
    X_te_tr = X_te_tr.drop(columns=cols)
    X_te_va = X_te_va.drop(columns=cols)
    X_te_te = X_te_te.drop(columns=cols)

    return X_te_tr, X_te_va, X_te_te

X_tr_xgb, X_va_xgb, X_te_xgb = oof_target_encode(X_tr, y_tr, X_va, X_te, cat_cols, n_splits=5, smoothing=20, add_noise=0.01)

dtr = xgb.DMatrix(X_tr_xgb, label=y_tr)
dva = xgb.DMatrix(X_va_xgb, label=y_va)
dte = xgb.DMatrix(X_te_xgb)

xgb_params = dict(
    objective='binary:logistic',
    eval_metric='auc',
    max_depth=8,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.1,
    reg_lambda=0.1,
    min_child_weight=1.0,
    tree_method='hist',
    seed=RANDOM_STATE,
)

xgb_model = xgb.train(
    xgb_params,
    dtr,
    num_boost_round=20000,
    evals=[(dtr,'train'), (dva,'valid')],
    early_stopping_rounds=300,
    verbose_eval=200
)

va_pred_xgb = xgb_model.predict(dva, iteration_range=(0, xgb_model.best_ntree_limit))
print("XGBoost AUC:", roc_auc_score(y_va, va_pred_xgb))
print("XGBoost PR-AUC:", average_precision_score(y_va, va_pred_xgb))


In [ ]:

# CatBoost требует индексы категориальных фичей относительно X_* столбцов
cat_idx = [X_tr.columns.get_loc(c) for c in cat_cols]

train_pool = Pool(data=X_tr, label=y_tr, cat_features=cat_idx)
valid_pool = Pool(data=X_va, label=y_va, cat_features=cat_idx)
test_pool  = Pool(data=X_te, cat_features=cat_idx)

cb_model = CatBoostClassifier(
    iterations=20000,
    learning_rate=0.05,
    depth=8,
    loss_function='Logloss',
    eval_metric='AUC',
    l2_leaf_reg=3.0,
    random_seed=RANDOM_STATE,
    od_type='Iter',
    od_wait=300,
    verbose=200
)

cb_model.fit(train_pool, eval_set=valid_pool, use_best_model=True)

va_pred_cb = cb_model.predict_proba(valid_pool)[:,1]
print("CatBoost AUC:", roc_auc_score(y_va, va_pred_cb))
print("CatBoost PR-AUC:", average_precision_score(y_va, va_pred_cb))


In [ ]:

from sklearn.metrics import roc_curve

scores = {
    'LightGBM': roc_auc_score(y_va, va_pred_lgb),
    'XGBoost':  roc_auc_score(y_va, va_pred_xgb),
    'CatBoost': roc_auc_score(y_va, va_pred_cb),
}
print("AUC scores:", scores)

# Простой бленд (усреднение)
va_blend = (va_pred_lgb + va_pred_xgb + va_pred_cb) / 3.0
print("Blend AUC:", roc_auc_score(y_va, va_blend))
print("Blend PR-AUC:", average_precision_score(y_va, va_blend))


In [ ]:

# Для корректности можно пересобрать модели на всей тренировочной части (или на полном train).
# Ниже — пример инференса на test с текущими моделями и усреднения предсказаний.

# LightGBM
te_pred_lgb = lgb_model.predict(X_te_lgb, num_iteration=lgb_model.best_iteration)

# XGBoost
te_pred_xgb = xgb_model.predict(dte, iteration_range=(0, xgb_model.best_ntree_limit))

# CatBoost
te_pred_cb = cb_model.predict_proba(test_pool)[:,1]

# Blend
te_blend = (te_pred_lgb + te_pred_xgb + te_pred_cb) / 3.0

sub = pd.read_csv(sub_path)
sub['isFraud'] = te_blend
out_path = "./ieee_blend_submission.csv"
sub.to_csv(out_path, index=False)
out_path
